In [1]:
from util.visualise import *

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
PAPER_DIR = str(Path.home())+"/papers/24-aaai-goose/paper"

PLOT_DIR = f"{PAPER_DIR}/plots"
PLOT_DIR

'/home/dillon/papers/24-aaai-goose/paper/plots'

In [4]:
L=4
aggr="max"

CONFIGS = ["blind", "hff", "shgn", "sdg-el dd", "fdg-el dd", "ldg-el dd", "sdg-el di", "fdg-el di", "ldg-el di"]
LINE_STYLES = [CONFIG_TO_LINE_STYLE[config] for config in CONFIGS]
COLOURS = [CONFIG_TO_COLOUR[config] for config in CONFIGS]

In [5]:
df = collect_test_stats_planner_and_graphs(CONFIGS, L, aggr, normalise=False)

In [6]:
# convert back to map then to latex
solved = []
df_solved = df.drop(columns=["expanded", "cost", "time"])
for domain in GOOSE_DOMAINS:
  df_domain_coverage = (df_solved.loc[df['domain'] == domain]).drop(columns=["domain"])
  df_domain_coverage = df_domain_coverage.groupby(['config'])['solved'].sum()
  df_list = df_domain_coverage.to_numpy()

  tmp = {}
  for i, config in enumerate(sorted(CONFIGS)):
    tmp[config] = df_list[i]
  solved_domain = []
  for config in CONFIGS:
    solved_domain.append(tmp[config])
  solved.append(solved_domain)

In [7]:
solved

[[0, 19, 0, 10, 11, 29, 14, 14, 16],
 [0, 90, 0, 33, 33, 78, 15, 6, 12],
 [1, 18, 0, 5, 9, 18, 4, 5, 9],
 [0, 36, 0, 10, 10, 1, 3, 3, 0],
 [74, 90, 0, 52, 56, 30, 48, 42, 37],
 [0, 0, 0, 0, 0, 55, 0, 0, 10],
 [0, 6, 0, 52, 35, 36, 44, 24, 38],
 [3, 26, 0, 78, 23, 3, 37, 14, 0]]

In [14]:
ret  = r"\begin{tabular}{"
for _ in range( len(CONFIGS)+1):
    ret += "c "
ret += r"} \toprule" + '\n'

ret += r"& \multicolumn{3}{c}{baseline} & \multicolumn{3}{c}{domain-dep.} & \multicolumn{3}{c}{domain-ind.} \\ \midrule" + '\n'

ret += "& "
for config in CONFIGS:
    ret += '\\header{' + CONFIG_TO_TEX[config] + '} & '
ret = ret[0:len(ret) - 2]
ret += r"\\ \midrule" + '\n'



for i, domain in enumerate(GOOSE_DOMAINS):
    ret += domain + f" ({len(os.listdir(f'../benchmarks/goose/{domain}/test'))}) & "
    ordered = sorted(solved[i], reverse=True)
    for count in solved[i]:
        if count == 0:
            LIME = ZERO
        elif count==ordered[0]:
            LIME = FIRST
        elif count==ordered[1]:
            LIME = SECOND
        elif count==ordered[2]:
            LIME = THIRD
        else:
            LIME = CELL
        ret += f"{LIME}{{{count}}}" + " & "
    ret = ret[0:len(ret) - 2]
    ret += r"\\" + '\n'


ret += r"\bottomrule\end{tabular}"

f = open(f"{PAPER_DIR}/tables/coverage.tex", 'w')
f.write(ret)
f.close()

In [9]:
df

,config,solved,expanded,time,cost,domain
0,blind,0,-1,-1.00,-1,blocks
1,hff,0,-1,-1.00,-1,blocks
2,shgn,0,-1,-1.00,-1,blocks
3,ldg-el dd,1,1211,44.82,138,blocks
4,ldg-el di,0,-1,-1.00,-1,blocks
...,...,...,...,...,...,...
3657,sdg-el dd,1,598,3.26,265,visitsome
3658,fdg-el dd,1,26779,106.07,1117,visitsome
3659,ldg-el dd,0,-1,-1.00,-1,visitsome
3660,sdg-el di,1,1078,5.13,281,visitsome


### Cumulative plots

In [10]:
df = collect_test_stats_planner_and_graphs(CONFIGS, L, aggr, normalise=True)

In [11]:
def plot_cum(df, x, x_label, save_file):
    ax = sns.ecdfplot(data=df, x=x, hue="config", hue_order=CONFIGS, stat="count", weights="solved")
    for lines, line_style, line_col, legend_handle in zip(ax.lines[::-1], LINE_STYLES, COLOURS, ax.legend_.legendHandles):
        lines.set_linestyle(line_style)
        lines.set_color(line_col)
        legend_handle.set_linestyle(line_style)
        legend_handle.set_color(line_col)
    plt.xscale("log")
    plt.xlabel(x_label)
    plt.ylabel('')
    if x=="time":
        plt.xlim(left=1)
    plt.tight_layout()
    # plt.show()
    plt.savefig(f"{PLOT_DIR}/{save_file.replace('.pdf','')}.pdf")
    plt.clf()

In [12]:
x = "time"
x_label = "time (s)"

for x, x_label in [("time", "time (s)"), ("expanded", "expanded nodes"), ("cost", "plan cost")]:
    plot_cum(df, x, x_label, f"cumulative-{x}-all")
    for domain in GOOSE_DOMAINS:
        df_domain = df.loc[df['domain'] == domain]
        plot_cum(df_domain, x, x_label, f"cumulative-{x}-{domain}")

<Figure size 432x288 with 0 Axes>